# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119121e+02     1.688845e+00
 * time: 0.3056619167327881
     1     1.075448e+01     9.780084e-01
 * time: 1.8552899360656738
     2    -1.157903e+01     1.095306e+00
 * time: 2.434138059616089
     3    -3.387834e+01     9.313897e-01
 * time: 3.2530999183654785
     4    -4.713904e+01     6.718415e-01
 * time: 4.0829918384552
     5    -5.680061e+01     2.350472e-01
 * time: 4.910333871841431
     6    -5.979454e+01     1.564275e-01
 * time: 5.461943864822388
     7    -6.092355e+01     4.648917e-02
 * time: 6.012775897979736
     8    -6.133581e+01     5.867481e-02
 * time: 6.571156024932861
     9    -6.158186e+01     3.525859e-02
 * time: 7.122318983078003
    10    -6.179908e+01     2.944372e-02
 * time: 7.674383878707886
    11    -6.197048e+01     2.856620e-02
 * time: 8.230557918548584
    12    -6.204796e+01     2.069395e-02
 * time: 8.78136396408081
    13    -6.211820e+01     1.554680e-02
 * time: 9.332489013671875
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557699
    AtomicNonlocal      14.8522667
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485373 
    Xc                  -19.3336820

    total               -62.261666458088
